# 집 근처 주유소 가격 확인
### 하는데 위도, 경도를 Googlemap이 아니라 Kakao에서 가져오기

In [2]:
import requests
from urllib.parse import quote
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [3]:
key_fd = open("./keys/kakaomapkey.txt", mode = "r", encoding = "utf8")
kmap_key = key_fd.read(100)
key_fd.close()

In [4]:
stations = pd.read_csv('04. stations.csv' , sep="," , encoding='utf8')
del stations['Unnamed: 0']

In [5]:
stations

,이름,주소,가격,셀프,상표,구
0,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1415.0,N,GS칼텍스,강동구
1,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),1415.0,Y,GS칼텍스,강동구
2,구천면주유소,서울 강동구 구천면로 357 (암사동),1424.0,N,현대오일뱅크,강동구
3,방아다리주유소,서울 강동구 동남로 811 (명일동),1435.0,Y,SK에너지,강동구
4,대성석유(주)직영 길동주유소,서울 강동구 천호대로 1168,1448.0,N,GS칼텍스,강동구
...,...,...,...,...,...,...
488,갤러리아주유소,서울 강남구 압구정로 426 (청담동),1848.0,N,SK에너지,강남구
489,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),1868.0,N,GS칼텍스,강남구
490,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),1869.0,N,GS칼텍스,강남구
491,(주)소모에너지엔테크놀러지 쎈트럴주유소,서울 강남구 삼성로 335 (대치동),1883.0,N,GS칼텍스,강남구


카카오에서 집 주소 가져오기

In [40]:
home_addr = '서울특별시 영등포구 당산로 214'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)

In [7]:
result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json()
result

{'documents': [{'address': {'address_name': '서울 영등포구 당산동5가 42',
    'b_code': '1156011500',
    'h_code': '1156056000',
    'main_address_no': '42',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_h_name': '당산2동',
    'region_3depth_name': '당산동5가',
    'sub_address_no': '',
    'x': '126.902834034778',
    'y': '37.5329312320335'},
   'address_name': '서울 영등포구 당산로 214',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 영등포구 당산로 214',
    'building_name': '당산 삼성 래미안',
    'main_building_no': '214',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_name': '당산동5가',
    'road_name': '당산로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.902834034778',
    'y': '37.5329312320335',
    'zone_no': '07214'},
   'x': '126.902834034778',
   'y': '37.5329312320335'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [8]:
result['documents'][0]['address']

{'address_name': '서울 영등포구 당산동5가 42',
 'b_code': '1156011500',
 'h_code': '1156056000',
 'main_address_no': '42',
 'mountain_yn': 'N',
 'region_1depth_name': '서울',
 'region_2depth_name': '영등포구',
 'region_3depth_h_name': '당산2동',
 'region_3depth_name': '당산동5가',
 'sub_address_no': '',
 'x': '126.902834034778',
 'y': '37.5329312320335'}

In [9]:
home_lat = float(result['documents'][0]['address']['y'])
home_lng = float(result['documents'][0]['address']['x'])

In [10]:
stations.index

RangeIndex(start=0, stop=493, step=1)

데이터프레임에 위도, 경도 추가하기
 - 안 나오는 값이 있어서 index에러 발생. nan값으로 바꾸고 처리

In [11]:
lat = [] ; lng = []

for i in tqdm(stations.index):
    try:
        addr = stations['주소'][i]
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)
        result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json()

        lat.append(float(result['documents'][0]['address']['y']))
        lng.append(float(result['documents'][0]['address']['x']))
    except:
        lat.append(np.nan)
        lng.append(np.nan)

stations['lat'] = lat
stations['lng'] = lng


100%|██████████| 493/493 [00:58<00:00,  8.44it/s]


In [44]:
stations.to_csv('05_stations.csv',sep=',',encoding='utf8')

In [59]:
stations = pd.read_csv('05_stations.csv')
del stations['Unnamed: 0']

- nan값 있는 행 빼고 가져오기

In [60]:
stations.isnull().any()

이름     False
주소     False
가격     False
셀프     False
상표     False
구      False
lat    False
lng    False
거리     False
dtype: bool

In [61]:
stations = stations[stations.lat.isnull() == False]
stations = stations[stations.lng.isnull() == False]

In [62]:
len(stations)

489

# 위도, 경도 간 거리재기

In [63]:
from haversine import haversine

In [64]:
home = (home_lat, home_lng)

In [65]:
stations['거리'] = stations.apply(lambda r: haversine(home, (r.lat,r.lng), unit='km'),axis=1)

In [66]:
stations['거리']

0      20.443404
1      23.615182
2      20.845347
3      22.324986
4      21.017290
         ...    
484    12.425625
485    15.881400
486    12.944388
487    14.218185
488    11.850029
Name: 거리, Length: 489, dtype: float64

In [67]:
stations.tail()

,이름,주소,가격,셀프,상표,구,lat,lng,거리
484,갤러리아주유소,서울 강남구 압구정로 426 (청담동),1848.0,N,SK에너지,강남구,37.526817,127.043532,12.425625
485,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),1868.0,N,GS칼텍스,강남구,37.498448,127.077572,15.881400
486,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),1869.0,N,GS칼텍스,강남구,37.511551,127.047115,12.944388
487,(주)소모에너지엔테크놀러지 쎈트럴주유소,서울 강남구 삼성로 335 (대치동),1883.0,N,GS칼텍스,강남구,37.501426,127.059074,14.218185
488,뉴서울(강남),서울 강남구 언주로 716 (논현동),2025.0,N,SK에너지,강남구,37.517669,127.035823,11.850029


# 3km 이내 비싼 곳 10곳, 싼 곳 10곳 가져오기

In [68]:
top10 = stations[stations['거리'] < 3].sort_values(by='가격',ascending=False).head(10)
top10

,이름,주소,가격,셀프,상표,구,lat,lng,거리
236,SJ오일(주) 여의도주유소,서울 영등포구 은행로 64,1939.0,N,GS칼텍스,영등포구,37.530376,126.925570,2.024884
235,KH여의도주유소,서울 영등포구 국회대로 794 (여의도동),1929.0,N,SK에너지,영등포구,37.530739,126.920299,1.559254
234,지에스칼텍스(주) 국회대로주유소,서울 영등포구 국회대로 746 (여의도동),1919.0,N,GS칼텍스,영등포구,37.527989,126.916029,1.286819
233,현대오일뱅크㈜직영 여의도현대주유소,서울 영등포구 의사당대로 154 (여의도동),1919.0,N,현대오일뱅크,영등포구,37.517680,126.927479,2.756699
232,버드나룻길주유소,서울 영등포구 버드나루로 111 (당산동),1890.0,N,SK에너지,영등포구,37.527974,126.907873,0.707988
231,sk여명주유소,서울 영등포구 국회대로62길 7 (여의도동),1688.0,N,SK에너지,영등포구,37.527612,126.916271,1.324295
56,SK에너지(주) 안국주유소,서울 마포구 양화로 33 (합정동),1584.0,N,SK에너지,마포구,37.549331,126.912557,2.015005
54,망원동주유소,서울 마포구 월드컵로 119 (망원동),1448.0,N,SK에너지,마포구,37.558741,126.907182,2.895413
230,대영주유소,서울 영등포구 국회대로 632 (영등포동7가),1430.0,Y,GS칼텍스,영등포구,37.525459,126.904114,0.838462
229,SK에너지(주) 양평주유소,서울 영등포구 선유로 195 (양평동3가),1418.0,Y,SK에너지,영등포구,37.531048,126.893507,0.848709


In [69]:
bot10 = stations[stations['거리'] < 3].sort_values(by='가격',ascending=True).head(10)
bot10

,이름,주소,가격,셀프,상표,구,lat,lng,거리
207,행촌주유소,서울 영등포구 도신로 130 (신길동),1319.0,N,SK에너지,영등포구,37.508497,126.905985,2.731197
208,(주)강서오일,서울 영등포구 도신로 151,1319.0,N,현대오일뱅크,영등포구,37.510015,126.908252,2.592597
209,매봉주유소,서울 영등포구 도신로 248 (신길동),1329.0,Y,현대오일뱅크,영등포구,37.513041,126.918333,2.600020
212,도림주유소,서울 영등포구 도림로 343 (도림동),1329.0,Y,알뜰주유소,영등포구,37.507770,126.900257,2.807040
214,㈜지에스이앤알 모드니주유소,서울 영등포구 경인로 789,1359.0,N,GS칼텍스,영등포구,37.514671,126.900728,2.038893
190,지에스칼텍스㈜ 건지주유소,서울 양천구 안양천로 1171 (목동),1365.0,Y,GS칼텍스,양천구,37.545316,126.882153,2.285081
215,보라주유소,서울 영등포구 영등포로 399 (신길동),1368.0,N,GS칼텍스,영등포구,37.513582,126.922725,2.775990
216,지에스칼텍스㈜ 화일주유소,서울 영등포구 도림로 415,1369.0,Y,GS칼텍스,영등포구,37.513257,126.896482,2.258271
192,(주)삼표에너지 목동주유소,서울 양천구 목동서로 129 (목동),1375.0,N,GS칼텍스,양천구,37.532164,126.875540,2.408229
193,목동주유소,서울 양천구 목동중앙로 125 (목동),1375.0,Y,SK에너지,양천구,37.539601,126.874553,2.601613


Folium으로 지도 그리기

In [ ]:
import folium

In [74]:
mapping = folium.Map( [home_lat, home_lng], zoom_start = 13)

# 아이콘 https://getbootstrap.com/docs/3.4/components/

# 집
folium.Marker( [home_lat, home_lng], 
    popup = home_addr , tooltip =  home_addr,
    icon=folium.Icon(color = 'blue', icon='home')).add_to(mapping)

folium.CircleMarker( [home_lat, home_lng] , radius = 15, color = 'white', fill_color = 'white').add_to(mapping)


for i in top10.index:
    folium.Marker( [top10.lat[i], top10.lng[i]], 
    popup = top10['주소'][i], tooltip = f"{top10['이름'][i]} 셀프: {top10['셀프'][i]}, {top10['가격']}",
    icon=folium.Icon(color = 'red', icon='remove')).add_to(mapping)

    folium.CircleMarker( [top10.lat[i], top10.lng[i]] , radius = 15, color = 'red', fill_color = 'purple').add_to(mapping)

for i in bot10.index:
    folium.Marker( [bot10.lat[i], bot10.lng[i]], 
    popup = bot10['주소'][i], tooltip =  f"{bot10['이름'][i]} 셀프: {bot10['셀프'][i]}, {bot10['가격']}",
    icon=folium.Icon(color = 'green', icon='heart')).add_to(mapping)

    folium.CircleMarker( [bot10.lat[i], bot10.lng[i]] , radius = 15, color = 'green', fill_color = 'blue').add_to(mapping)

mapping


In [73]:
# 한글이 나오기도 하고 안 나오기도 함
mapping.save('집 주변 주유소.html')